<a href="https://colab.research.google.com/github/Mrxiba/FAFDG/blob/main/MIMO_multi_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Finish Single MIMO Here ==============================================================
=============================================================
do here

In [ ]:
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install -q flwr[simulation] flwr_datasets[vision] torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.7/364.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 953.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl
from flwr.common import Metrics

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.3.0+cu121 and Flower 1.9.0


In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

#NUM_CLIENTS = 2
BATCH_SIZE = batch_size = 50
ngpu = 0
dataroot_Inline = "/content/Inline.csv"
#dataroot_Passive = "/content/Passive.csv"
#dataroot_vFW = "/content/vFW.csv"
#dataroot_test = "/content/vFW.csv"

class VNFDataset(Dataset):
    def __init__(self, root):
        self.df = pd.read_csv(root)
        self.data = self.df.to_numpy()
        labels=['CPUUTP','MEMUTP','RTT']
        predictors = list(set(list(self.df.columns))-set(labels))
        self.df[predictors] = self.df[predictors]/self.df[predictors].max()# guiyihua
        self.df[predictors].dropna(axis=0)
        self.df[labels].dropna(axis=0)
        self.x = torch.tensor(self.df[predictors].values, dtype=torch.float32)
        self.y = torch.tensor(self.df[labels].values, dtype=torch.float32)
    def __getitem__(self, idx):
        return self.x[idx, :], self.y[idx,:]
    def __len__(self):
        return len(self.data)

def load_datasets():

    Inline_data = VNFDataset(dataroot_Inline)

    #Passive_data = VNFDataset(dataroot_Passive)

    #vFW_data = VNFDataset(dataroot_vFW)

    testset = Inline_data

    # Split training set into 2
    datasets = [Inline_data]

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 30  # 30 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=True))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE,shuffle=True)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets()

In [ ]:
images, labels = next(iter(trainloaders[0]))

In [ ]:
images[49].shape

torch.Size([6])

In [ ]:
labels[49].shape

torch.Size([3])

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(6, 24),
            nn.ReLU(),
            nn.Linear(24, 12),
            nn.ReLU(),
            nn.Linear(12, 6),
            nn.ReLU(),
            nn.Linear(6, 3)
        )

    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x

In [ ]:
Net63=Net()
print("The parameters: ", list(Net63.parameters()))

The parameters:  [Parameter containing:
tensor([[-0.1414, -0.0557, -0.1621, -0.0435, -0.1092,  0.1181],
        [ 0.0087,  0.2022,  0.2812, -0.3872,  0.0265, -0.0775],
        [-0.3072,  0.2554, -0.1521,  0.1040,  0.1152,  0.3605],
        [-0.1264, -0.3635, -0.3807,  0.1837,  0.1222, -0.3856],
        [-0.3135,  0.1158,  0.1290, -0.3334,  0.2260,  0.1822],
        [ 0.1010, -0.2609,  0.1565, -0.2776,  0.2265,  0.1687],
        [-0.3695,  0.2422, -0.1366, -0.0756,  0.3044,  0.2737],
        [ 0.2182,  0.2485, -0.0861,  0.2632,  0.2743, -0.3689],
        [-0.0829, -0.3823, -0.3611, -0.0983, -0.1653,  0.0413],
        [-0.2737,  0.0339,  0.1935, -0.1914, -0.0974,  0.1244],
        [ 0.1558,  0.1622,  0.0928,  0.2761,  0.2678, -0.4003],
        [ 0.0200,  0.2564, -0.3420,  0.3919, -0.3068, -0.1743],
        [ 0.3735,  0.1217,  0.0731,  0.3364,  0.0061,  0.0133],
        [-0.3875,  0.0813, -0.1398,  0.2411,  0.0145,  0.2290],
        [ 0.2616, -0.1590,  0.1943,  0.1149, -0.1145,  0.3059],


In [ ]:
import torch
from torcheval.metrics import MeanSquaredError
from torcheval.metrics import R2Score


def train(net, trainloader, epochs: int,verbose=True):
    """Train the network on the training set."""
    criterion = torch.nn.MSELoss() #MSE
    L1 = torch.nn.L1Loss() #MAE
    metricR2 = R2Score()#R2
    optimizer = torch.optim.Adam(net.parameters(),lr=0.001)
    net.train()
    for epoch in range(epochs):
      epoch_loss, epoch_loss2, epoch_loss3 = 0.0, 0.0, 0.0
      for images, labels in trainloader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = net(images)
        labels = labels.float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # Metrics
        epoch_loss = L1(outputs, labels)#MAE loss
        epoch_loss2 = torch.sqrt(loss) #MSE loss
        epoch_loss3 = metricR2.update(outputs, labels).compute().item()
      if verbose:
        print(f"Epoch {epoch+1}: train MAE {epoch_loss}, train RMSE {epoch_loss2}, train R2 {epoch_loss3}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    MAE,RMSE, R2 = 0.0, 0.0, 0.0
    net.eval()
    L1 = torch.nn.L1Loss() #MAE
    metricMSE = MeanSquaredError()#MSE
    metricR2 = R2Score()#R2
    with torch.no_grad():
        for images, labels in testloader:
          images, labels = images.to(DEVICE), labels.to(DEVICE)
          outputs = net(images)
          labels = labels.float()
          # Metrics
          MAE = L1(outputs, labels)
          RMSE = torch.sqrt(metricMSE.update(outputs, labels).compute()).item()
          R2 = metricR2.update(outputs, labels).compute().item()

    return MAE, RMSE, R2

In [ ]:
def test_MIMO(net, testloader):
    """Evaluate the network on the entire test set."""
    MAE_CPUUTP,RMSE_CPUUTP, R2_CPUUTP = 0.0, 0.0, 0.0
    MAE_MEMUTP,RMSE_MEMUTP, R2_MEMUTP = 0.0, 0.0, 0.0
    MAE_RTT,RMSE_RTT, R2_RTT = 0.0, 0.0, 0.0
    net.eval()
    L1 = torch.nn.L1Loss() #MAE
    metricMSE = MeanSquaredError()#MSE
    metricR2 = R2Score()#R2
    with torch.no_grad():
        for images, labels in testloader:
          images, labels = images.to(DEVICE), labels.to(DEVICE)
          outputs = net(images)
          labels = labels.float()

          # Metrics
          MAE_CPUUTP = L1(outputs[:,0], labels[:,0])
          RMSE_CPUUTP = torch.sqrt(metricMSE.update(outputs[:,0], labels[:,0]).compute()).item()
          R2_CPUUTP = metricR2.update(outputs[:,0], labels[:,0]).compute().item()

          MAE_RTT = L1(outputs[:,2], labels[:,2])
          RMSE_RTT = torch.sqrt(metricMSE.update(outputs[:,2], labels[:,2]).compute()).item()
          R2_RTT = metricR2.update(outputs[:,2], labels[:,2]).compute().item()

    return MAE_CPUUTP, RMSE_CPUUTP, R2_CPUUTP, MAE_RTT, RMSE_RTT, R2_RTT

In [ ]:
trainloader = trainloaders[0]
valloader = valloaders[0]
net = Net().to(DEVICE)

for epoch in range(600):
    train(net, trainloader, 1)
    loss_1, loss_2, loss_3 = test(net, valloader)
    print(f"Epoch {epoch+1}: val all MAE {loss_1}, val all RMSE {loss_2}, val all R2 {loss_3}")
    MAE_CPUUTP, RMSE_CPUUTP, R2_CPUUTP, MAE_RTT, RMSE_RTT, R2_RTT = test_MIMO(net, valloader)
    print(f"Epoch {epoch+1}: val CPUUTP MAE {MAE_CPUUTP}, val CPUUTP RMSE {RMSE_CPUUTP}, val CPUUTP R2 {R2_CPUUTP}")
    print(f"Epoch {epoch+1}: val RTT MAE {MAE_RTT}, val RTT RMSE {RMSE_RTT}, val RTT R2 {R2_RTT}")

MAE_e, RMSE_e, R2_e = test(net, testloader)
print(f"Final test set performance:\n \ testMAE {MAE_e}\n \ testRMSE {RMSE_e}\n \ testR2 {R2_e}")
t_MAE_CPUUTP, t_RMSE_CPUUTP, t_R2_CPUUTP, t_MAE_RTT, t_RMSE_RTT, t_R2_RTT = test_MIMO(net, testloader)
print(f"Final test set CPUUTP performance:\n \ test_CPUUTP_MAE {t_MAE_CPUUTP}\n \ test_CPUUTP_RMSE {t_RMSE_CPUUTP}\n \ test_CPUUTP_R2 {t_R2_CPUUTP}")
print(f"Final test set RTT performance:\n \ test_RTT_MAE {t_MAE_RTT}\n \ test_RTT_RMSE {t_RMSE_RTT}\n \ test_RTT_R2 {t_R2_RTT}")
#print(f"Final test set CPUUTP performance:\n\test_CPUUTP_MAEloss {t_MAE_CPUUTP}\n\test_CPUUTP_RMSEloss {t_RMSE_CPUUTP}\n\test_CPUUTP_R2loss {t_R2_CPUUTP}")

<>:14: DeprecationWarning: invalid escape sequence '\ '


Epoch 1: train MAE 1.1255629062652588, train RMSE 1.3002777099609375, train R2 -67.4964599609375
Epoch 1: val all MAE 1.129528522491455, val all RMSE 1.2965812683105469, val all R2 -126.4278564453125
Epoch 1: val CPUUTP MAE 1.2939703464508057, val CPUUTP RMSE 1.2965636253356934, val CPUUTP R2 -287.7788391113281
Epoch 1: val RTT MAE 1.4012008905410767, val RTT RMSE 1.5094751119613647, val RTT R2 -3.5838160514831543
Epoch 1: train MAE 1.0296216011047363, train RMSE 1.2144415378570557, train R2 -54.96572494506836
Epoch 2: val all MAE 1.003760576248169, val all RMSE 1.1790509223937988, val all R2 -101.26528930664062
Epoch 2: val CPUUTP MAE 1.1806873083114624, val CPUUTP RMSE 1.184464693069458, val CPUUTP R2 -239.99417114257812
Epoch 2: val RTT MAE 1.2523231506347656, val RTT RMSE 1.383949637413025, val RTT R2 -2.8531482219696045
Epoch 1: train MAE 0.7656921744346619, train RMSE 0.9705546498298645, train R2 -38.75201416015625
Epoch 3: val all MAE 0.8003180027008057, val all RMSE 1.001341342

======================================================= Code end here

Ignore the underneath context
# **Ignore the underneath**

In [ ]:
L1 = torch.nn.L1Loss()
metricMSE = MeanSquaredError()#MSE
metricR2 = R2Score()#R2


MAE_CPUUTP = L1(outputs[:,0], labels[:,0])
MSE_CPUUTP = metricMSE.update(outputs[:,0], labels[:,0]).compute().item()
R2_CPUUTP = metricR2.update(outputs[:,0], labels[:,0]).compute().item()

print(f"Final test set metrics:\n \\ pytorchMAEloss {MAE_CPUUTP}\n\\ pytorchMSEloss {MSE_CPUUTP}\n\\ pytorchR2 {R2_CPUUTP}")

Final test set metrics:
 \ pytorchMAEloss 0.052874356508255005
\ pytorchMSEloss 0.006991443689912558
\ pytorchR2 0.4459463953971863


In [ ]:
a = outputs[:,0] #c
b= labels[:,0] #d

In [ ]:
f= torch.sqrt(a)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import sklearn.metrics as metrics
mae_poly = metrics.mean_absolute_error(d, c)
mse_poly = metrics.mean_squared_error(d, c)
rmse_poly = np.sqrt(mse_poly)
r2_poly = metrics.r2_score(d,c)
print(f"MAE: {mae_poly}")
print(f"MSE: {mse_poly}")
print(f"RMSE: {rmse_poly}")
print(f"R2: {r2_poly}")

MAE: 0.052874000000000004
MSE: 0.006990763
RMSE: 0.08361078279743588
R2: 0.44599349962667323
